In [1]:
import torch
import torch.nn.functional as F

# 입력 행렬 (6x6 크기)
input_matrix = torch.tensor([
    [1, 1, 1, 1, 1, 1],
    [2, 2, 2, 2, 2, 2],
    [4, 4, 4, 4, 4, 4],
    [2, 2, 2, 2, 2, 2],
    [1, 1, 1, 1, 1, 1],
    [2, 2, 2, 2, 2, 2]
], dtype=torch.float32)

# 커널 행렬 (3x3 크기)
kernel_matrix = torch.tensor([
    [1, 0.5, 1],
    [-1, -0.5, 1],
    [1, 1, 0.5]
], dtype=torch.float32)
# -0.5 = ff80
# -1 = ff00
# 입력과 커널을 Conv2d에 맞게 (N, C, H, W) 형식으로 변환
input_tensor = input_matrix.unsqueeze(0).unsqueeze(0)  # (1, 1, 6, 6)
kernel_tensor = kernel_matrix.unsqueeze(0).unsqueeze(0)  # (1, 1, 3, 3)

# Convolution 연산 (패딩=1)
result = F.conv2d(input_tensor, kernel_tensor, padding=1)

# 결과 출력
print("Padding=1 적용한 단순 Convolution 결과:")
print(result.squeeze().numpy())


Padding=1 적용한 단순 Convolution 결과:
[[ 3.5  4.5  4.5  4.5  4.5  2.5]
 [ 8.5 11.5 11.5 11.5 11.5  6.5]
 [ 8.   8.   8.   8.   8.   1. ]
 [ 8.5 11.5 11.5 11.5 11.5  5. ]
 [ 6.5  9.5  9.5  9.5  9.5  5.5]
 [ 2.5  1.5  1.5  1.5  1.5 -1.5]]


In [14]:
import torch
import torch.nn.functional as F

# 6x6 크기의 0부터 35까지의 값으로 구성된 입력 행렬 생성
input_matrix = torch.arange(36, dtype=torch.float32).reshape(1, 1, 6, 6)

# 첫 번째 값을 -1로 설정
input_matrix[0, 0, 0, 0] = -1

print("입력 행렬 (첫 번째 값 -1, 나머지 0부터 35까지):")
print(input_matrix.squeeze().numpy())

# Max Pooling 연산 (커널 크기=2, 스트라이드=2)
pooled_output = F.max_pool2d(input_matrix, kernel_size=2, stride=2)

print("\nMax Pooling 결과 (커널=2, 스트라이드=2):")
print(pooled_output.squeeze().numpy())


입력 행렬 (첫 번째 값 -1, 나머지 0부터 35까지):
[[-1.  1.  2.  3.  4.  5.]
 [ 6.  7.  8.  9. 10. 11.]
 [12. 13. 14. 15. 16. 17.]
 [18. 19. 20. 21. 22. 23.]
 [24. 25. 26. 27. 28. 29.]
 [30. 31. 32. 33. 34. 35.]]

Max Pooling 결과 (커널=2, 스트라이드=2):
[[ 7.  9. 11.]
 [19. 21. 23.]
 [31. 33. 35.]]


In [ ]:
import numpy as np

# 입력 데이터 (1x4 벡터)
input_data = np.array([1, 2, 3, 4])

# 가중치 행렬 (4x2 행렬)
weights = np.array([
    [1, 2],
    [2, 3],
    [4, 5],
    [6, 7]
])

# Dense 층 연산 (입력과 가중치의 행렬 곱)
output = np.dot(input_data, weights)

print("Dense 층 출력:")
print(output)


<h1> to fixed point 

In [ ]:
import numpy as np

def fixed_point_hex(num, frac_bits=8, total_bits=16):
    scale = 1 << frac_bits  # Scale factor: 2^8 = 256
    fixed_num = int(round(num * scale))
    
    # Check for overflow
    max_int = (1 << (total_bits - 1)) - 1
    min_int = - (1 << (total_bits - 1))
    if fixed_num > max_int or fixed_num < min_int:
        raise OverflowError("Value out of range for fixed-point representation.")

    if fixed_num >= 0:
        hex_value = '{:04X}'.format(fixed_num)
    else:
        # Compute 2's complement for negative numbers
        fixed_num_twos_complement = (1 << total_bits) + fixed_num
        hex_value = '{:04X}'.format(fixed_num_twos_complement)
    return hex_value


# Vectorize the function to apply it element-wise to the numpy array
fixed_point_hex_vectorized = np.vectorize(fixed_point_hex)

# Convert the image array to fixed-point hexadecimal values
hex_origin_image = fixed_point_hex_vectorized(input_matrix)
hex_image = fixed_point_hex_vectorized(result)
hex_kernel = fixed_point_hex_vectorized(kernel_matrix)
# Print the resulting hexadecimal values in the desired format
print("* original image (fixed)")
for row in hex_origin_image:
    print(row)

print("* processed image (fixed)")
for row in hex_image:
    for value in row:
        print(value)


print("* Kernel (fixed)")
for row in hex_kernel:
    for value in row:
        print(value)


* original image (fixed)
0100
0100
0100
0100
0100
0100
0200
0200
0200
0200
0200
0200
0400
0400
0400
0400
0400
0400
0200
0200
0200
0200
0200
0200
0100
0100
0100
0100
0100
0100
0200
0200
0200
0200
0200
0200
* processed image (fixed)
[['0380' '0480' '0480' '0480' '0480' '0280']
 ['0880' '0B80' '0B80' '0B80' '0B80' '0680']
 ['0800' '0800' '0800' '0800' '0800' '0100']
 ['0880' '0B80' '0B80' '0B80' '0B80' '0500']
 ['0680' '0980' '0980' '0980' '0980' '0580']
 ['0280' '0180' '0180' '0180' '0180' 'FE80']]
* Kernel (fixed)
0100
0080
0100
FF00
FF80
0100
0100
0100
0080
